In [ ]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
import cv2
import glob

from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
from tensorflow import keras as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalMaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
xception_model = K.applications.Xception(
    weights='imagenet', 
    input_shape=(200, 200, 3),
    include_top=False)
x = xception_model.output
x = GlobalMaxPooling2D()(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=xception_model.input, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
#Replace filename with the trained model to use
model.load_weights('FILENAME.h5')

In [ ]:
#The GradCam implementation is based on paper Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization

def heatmap(dirname):
    count = 0
    for img in glob.glob(f"{dirname}/*.jpg"):
        face_org = cv2.imread(img)
        face_res = cv2.resize(face_org, (200,200))
        img = image.load_img(img, target_size=(200, 200))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        preds = model.predict(x=x,
        verbose=1,
        steps=None,
        callbacks=None,
        max_queue_size=10,
        workers=1,
        use_multiprocessing=False,
        )
        crop_output = model.output[:, 0]

        last_conv_layer = model.get_layer('block14_sepconv1')

        grads = K.backend.gradients(crop_output, last_conv_layer.output)[0]
    
        pooled_grads = K.backend.mean(grads, axis=(0, 1, 2))

        iterate = K.backend.function([model.input], [pooled_grads, last_conv_layer.output[0]])

        pooled_grads_value, conv_layer_output_value = iterate([x])

        for i in range(512):
            conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

        heatmap = np.mean(conv_layer_output_value, axis=-1)
        heatmap = np.maximum(heatmap, 0)
        heatmap /= np.max(heatmap)
  
        img = face_org

        heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

        heatmap = np.uint8(255 * heatmap)

        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

        superimposed_img = heatmap * 0.4 + img
        
#Replace path with path to folder where to store the overlay images
        cv2.imwrite('WRITE PATH/%imposed.jpg' % count, superimposed_img)
        count += 1

In [ ]:
#Replace path with the path to source folder
heatmap('SOURCE PATH')